# Statistics of the European XFEL


Free Electron Lasers are parametric amplifiers that actively filter the temporal and spatial frequencies of the input synchrotron radiation signal during a high-gain amplification process. Two regimes of the amplification can be identified depedening on the saturation of the electron beam. 

Prior to saturation, the amplification process is linear. We define linear-regime corresponds to the propagation distances over which $z$ < $z_{sat}$, where $z_{sat}$ is the saturation point which we define as the point where the amplification is no-longer exponential. In the case of SASE FELs, the input signal is shot-noise fluctuations in the electron beam current density which originate from Poisson processes in photo-injection, which can be approximated to be a Gaussian random process. A linearly filtered Gaussian random process is also Gaussian, therefore in the linear-regime, the field inherents the statistical properties of the beam-current. For a Gaussian process, all higher-order correlation functions can be determined from a single measurement of the correlation of the field. We therefore expect that the probability distribution of the instantaneous radiation will follow a negative-exponential, and that the integrals of the instantaeous intensity and the integrated spectral density will fluctuate in accordance with the Gamma distribution:

$
p(W) = \frac{M^M}{\Gamma(M)}\left(\frac{W}{\langle W \rangle}\right)^{M-1}\frac{1}{\langle W \rangle} \exp\left(-M\frac{W}{\langle W \rangle} \right)
$

In the non-linear regime...


In [1]:
from felpy.model.source import SA1_Source
from felpy.model.mesh import Mesh
import numpy as np
from tqdm import tqdm

In [4]:
mesh = Mesh(nx = 512, ny = 512, xMin = -300e-06, xMax = 300e-06, yMin= -300e-06, yMax= 300e-06)

gm = SA1_Source(ekev = 4.98, q = 0.25, S = 10, mesh = mesh)

def get_temporal_profiles(source, n_spectra = 10, **kwargs):
        """ 
        Return the temporal profiles of a statistical source
        """
        temporal_profiles = np.zeros([source.nz, n_spectra])
        
        for itr in tqdm(range(n_spectra)):
            temporal_profiles[:, itr] = source.get_temporal_profile(refresh = True, sigma = 3)
            
        return temporal_profiles

temporal_profiles = get_temporal_profiles(gm, n_spectra = 2500000)


100%|██████████| 2500000/2500000 [05:14<00:00, 7952.48it/s]


In [ ]:
from felpy.utils.vis_utils import Grids, animate
from felpy.analysis.statistics.correlation import norm 


for itr in tqdm(range(0,temporal_profiles.shape[1]-1)):
    
    if itr % 1000 == 0:
        grid = Grids(scale = 2, global_aspect = 2.25)
        grid.create_grid(n = 1, m = 2, sharex = False, sharey = True)

        grid.set_fontsize(22)
        grid.pad(2)

        ax1, ax2 = grid.axes.flatten()
        #temporal_profiles.shape

        from matplotlib import pyplot as plt
        from numpy.fft import ifft, fftshift

        ti = abs(temporal_profiles)**2
        ts = abs((ifft(temporal_profiles, axis = -1)))**2

        t = np.linspace(-gm.pulse_duration/2, gm.pulse_duration/2, gm.nz)
        w = np.linspace(-1/gm.pulse_duration/2, 1/gm.pulse_duration/2, gm.nz)

        ax1.plot(t*1e15, norm(ti[:,itr]), alpha = 0.25, label = r'$f_n(t)u(t)$')
        ax1.plot(t*1e15, norm(ti[:,:itr].mean(-1)), label = r'$\langle f_n(t)u(t)\rangle$')
        ax1.text(np.min(t*1e15), 0.95, 'n = {}'.format(itr+1), size=18, ha='left', va='center')
        ax1.legend(fontsize = 18, loc = 'upper right')
        ax1.set_ylabel("Normalised Intensity (a.u)")
        ax2.set_xlabel("$\omega$ (1/s)")
        from felpy.model.source import gaussian_profile 

        ax2.plot(w, norm(ts[:,itr]), alpha = 0.25, label = r'$\tilde{f}_n(\omega)\tilde{g}(\omega)$')
        ax2.plot(w, norm(ts[:,:itr].mean(-1)), label = r'$\langle \tilde{f}(\omega)\tilde{g}(\omega) \rangle$')

        ax2.legend(fontsize = 18, loc = 'upper right')
        ax2.text(np.min(w), 0.95, 'n = {}'.format(itr), size=18, ha='left', va='center')
        ax1.set_xlabel("Time (fs)")

        grid.savefig("./images/slow_gifs/env_plot_{:06d}.png".format(itr))


animate("./images/slow_gifs/", "./images/", "envelope_long", delay = 0.05, rmdir = False)


  1%|          | 24001/2499999 [17:31<43:40:42, 15.75it/s]  